<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Arabic/biGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arabic Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Arabic/

/content/drive/My Drive/dataset/OffenseEval2020/data/Arabic


In [3]:
ls

offenseval-ar-dev-v1.tsv  offenseval-ar-training-v1.tsv  readme-data-ar.txt


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-ar-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [5]:
train.head()

,id,tweet,subtask_a
2484,2491,@USER كفو يا بن زنان يا صليب الراس يالعالمي يا...,NOT
4030,4102,يا فهد باص يا فهد باص يا فهد باص يا فهد باص يا...,OFF
2455,2462,يا نفس يا ولهانه <LF>لي ما درى بك حد <LF>اتحمل...,NOT
2987,3059,- بكام الشيميز ده لو سمحت ؟<LF>= 420 جنية يا ف...,NOT
3112,3184,يالله اللحين يا فقراري يا ظريف يا ملك التلاعب ...,OFF


In [6]:
train.tail()

,id,tweet,subtask_a
3826,3898,RT @USER: يا وهّاب يا باسط.. <LF>يُغالبني القل...,NOT
5984,6082,والله يا نوجا يا حبيبت قلبي كنت زي القمر جنب ا...,NOT
2118,2119,@USER يا بكاااايه يا بكاااايه اسكت ازعجتنا ابك...,NOT
6043,6141,@USER كل سنه وانت طيب يا باور يا عالمى وعقبال ...,NOT
4246,4318,#السيسى_رئيسى_وأفتخر<LF>#السيسى_زعيمى_وأفتخر <...,NOT


In [0]:
val=pd.read_csv( 'offenseval-ar-dev-v1.tsv',sep="\t")

In [0]:
train_labels=train['subtask_a']
X_train=train['tweet']
y_train=pd.factorize(train_labels)[0]

In [9]:
import collections
collections.Counter(train_labels)

Counter({'NOT': 5468, 'OFF': 1371})

In [0]:
val_labels=val['subtask_a']
X_val=val['tweet']
y_val=pd.factorize(val_labels)[0]

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(X_train)
sequences = tokenize.texts_to_sequences(X_train)
word_index = tokenize.word_index
X_train_sequences = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [12]:
len(X_train_sequences),len(train_labels)

(6839, 6839)

In [0]:

val_sequences = tokenize.texts_to_sequences(X_val)
X_val_sequences = sequence.pad_sequences(val_sequences,maxlen=max_len)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
model = Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=max_len))
model.add(Bidirectional(CuDNNGRU(150, return_sequences = True)))
#model.add(Bidirectional(CuDNNGRU(50, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dense(20, activation="relu"))

model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [19]:

model.fit(X_train_sequences,y_train,batch_size=256,epochs=10,verbose=2,class_weight=class_weights,validation_data=(X_val_sequences,y_val),callbacks=[reduce_lr])

Train on 6839 samples, validate on 1000 samples
Epoch 1/10
 - 6s - loss: 0.6577 - f1: 0.4374 - val_loss: 0.5546 - val_f1: 0.5666
Epoch 2/10
 - 2s - loss: 0.4447 - f1: 0.6909 - val_loss: 0.3759 - val_f1: 0.6604
Epoch 3/10
 - 2s - loss: 0.1621 - f1: 0.8744 - val_loss: 0.3160 - val_f1: 0.6785
Epoch 4/10
 - 2s - loss: 0.0658 - f1: 0.9562 - val_loss: 0.4039 - val_f1: 0.6555
Epoch 5/10
 - 2s - loss: 0.0300 - f1: 0.9805 - val_loss: 0.4295 - val_f1: 0.6469

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 6/10
 - 2s - loss: 0.0150 - f1: 0.9926 - val_loss: 0.4399 - val_f1: 0.6416
Epoch 7/10
 - 2s - loss: 0.0116 - f1: 0.9933 - val_loss: 0.4632 - val_f1: 0.6446

Epoch 00007: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 8/10
 - 2s - loss: 0.0103 - f1: 0.9932 - val_loss: 0.4671 - val_f1: 0.6446
Epoch 9/10
 - 2s - loss: 0.0103 - f1: 0.9938 - val_loss: 0.4692 - val_f1: 0.6500

Epoch 00009: ReduceLROnPlateau reducing learning rate to 8

In [22]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_val_sequences, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

1000/1000 [==============================] - 0s 499us/step
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       821
           1       0.65      0.65      0.65       179

    accuracy                           0.88      1000
   macro avg       0.79      0.79      0.79      1000
weighted avg       0.88      0.88      0.88      1000



In [0]:
import keras
keras.backend.clear_session()